#### Brittani Borroto_ANA680_Assignment 2_Design a Three-Layered ANN Classifier

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
#load dataset
df = pd.read_csv("Churn_Modelling.csv")

In [14]:
#dataset review
print(df.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

In [16]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB
None


In [17]:
print(df.isnull().sum())

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64


#

##### Dataset review:

This Bank Customer Churn dataset contains information on 10,000 customers, with 13 features and 1 target label indicating whether the customer has exited / churned or the customer remained with the bank. This dataset has no missing values. Each row represents one customer, and the Exited column indicates churn status or zero if the customer stayed and one if the customer exited / churned. 


#


1.	RowNumber: Sequential index for each customer (not predictive)
  
3.	CustomerId: Unique customer identifier (not predictive)
  
5.	Surname: Customer’s last name (not predictive)
  
7.	CreditScore: Integer score representing creditworthiness (about 350–850)
  
9.	Geography: Country of residence (France, Spain, Germany)
   
11.	Gender: Customer gender (Male, Female)
   
13.	Age: Customer age in years (young adults to seniors)
   
15.	Tenure: Number of years the customer has been with the bank (0–10)
   
17.	Balance: Account balance in dollars (zero for high-income customers)
   
19.	NumOfProducts: Number of bank products the customer uses (1–4)
   
21.	HasCrCard: Binary flag indicating if the customer has a credit card (1 = yes, 0 = no)
   
23.	IsActiveMember: Binary flag indicating if the customer is actively engaged (1 = active, 0 = inactive)
   
25.	EstimatedSalary: Annual salary estimate in dollars
   
27.	Exited: Target variable — 1 if the customer churned, 0 if retained


#



In [3]:
#create target variable
y = df["Exited"]

In [4]:
#feature selection (drop irrelevant columns)
X = df.drop(columns=["RowNumber", "CustomerId", "Surname", "Exited"])

In [5]:
#encode categorical variables,  male=1, female=0 with one-hot encode 
X["Gender"] = LabelEncoder().fit_transform(X["Gender"])  
X = pd.get_dummies(X, columns=["Geography"], drop_first=True)  

In [6]:
#feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:
#ANN
model = Sequential()

#use ‘relu’ activation for the first and second layers
model.add(Dense(units=6, activation='relu', input_dim=X_train.shape[1]))

#use ‘relu’ activation for the first and second layers
model.add(Dense(units=6, activation='relu'))

#'sigmoid’ for the last dense layer.
model.add(Dense(units=1, activation='sigmoid'))

#for compiling, use ‘adam’ optimizer and loss should be ‘binary_crossentropy’
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\britt\OneDrive\Documents\M_ANA615\ANACONDA\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
#train ANN
model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=0)

In [10]:
#predict on test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [11]:
#confusion matrix and accuracy
cm = confusion_matrix(y_test, y_pred_binary)
acc = accuracy_score(y_test, y_pred_binary)

print("Confusion Matrix:\n", cm)
print("Accuracy:", round(acc, 4))

Confusion Matrix:
 [[1537   70]
 [ 206  187]]
Accuracy: 0.862


#

This ANN model correctly predicted 86.2% of cases overall. Though is model is better at precision than recall, I would proceed with caution about predicting churn, as this ANN model missed many actual churners and this can become costly in business terms. 


TN = 1537 : the ANN model correctly identified 1537 customers who did not churn

FP = 70: the ANN model identified 70 customers who were predicted to churn but actually stayed

FN = 206: the ANN model missed 206 churners, this is critical in churn modeling

TP = 187: the ANN model correctly identified correctly identified 187 churners



#



##### [[TN, FP]
##### [FN, TP]]


##### Legend:

TN = True Negative: Customers who stayed and were correctly predicted as staying

FP = False Positives: Customers who stayed but were incorrectly predicted to churn

FN = False Negatives: Customers who churned but were missed by the model

TP = True Positives: Customers who churned and were correctly predicted to churn


#